# 01 – Exploração do Snapshot

Primeiro notebook do **creditlab**.

Nele, vamos:
1. Instanciar o `CreditDataSynthesizer`.
2. Gerar a *safra 0* (snapshot) para alguns grupos homogêneos.
3. Fazer uma exploração inicial: contagem de contratos por grupo, estatísticas descritivas e um histograma de dias em atraso.

⚠️ Este notebook assume que o módulo `credit_data_synthesizer.py` está no mesmo diretório ou em `PYTHONPATH`.

In [18]:
import sys
from pathlib import Path

# Adiciona o diretório-pai ao sys.path
sys.path.append(str(Path().resolve().parent))

import pandas as pd
import matplotlib.pyplot as plt


from importlib import reload
import credit_data_synthesizer as cds
reload(cds)  # garante versão atual no notebook

from credit_data_synthesizer import (
    CreditDataSynthesizer,
    default_group_profiles,   # função helper já incluída no módulo
)

from credit_data_sampler import (
    TargetSampler
)

## Parâmetros iniciais

In [19]:
# from credit_data_synthesizer import CreditDataSynthesizer, default_group_profiles
# from credit_data_sampler import TargetSampler

# # gerar painel com churn dinamico
# synth = CreditDataSynthesizer(
#     group_profiles=default_group_profiles(2),
#     contracts_per_group=1_000,
#     n_safras=12,
#     new_contract_rate=0.05,
#     closure_rate=0.03,
# )
# _, panel, _ = synth.generate()

# # rebalancear bad-rate preservando ranking dos grupos
# sampler = TargetSampler(target_ratio=0.08, preserve_rank=True)
# panel_bal = sampler.fit_transform(panel, target_col="ever90m12", safra_col="safra")
# display(panel_bal.head())

In [20]:
#panel_bal.groupby('grupo_homogeneo')['ever90m12'].mean()*100

In [21]:
# ----- parâmetros que você quer -----
n_groups = 5
contracts_per_group = 1_000
n_safras = 24
seed = 0

# 1) Obtenha os perfis padrão e pegue só os três primeiros
group_profiles = default_group_profiles(n_groups)

# 2) Instancie o sintetizador
synth = CreditDataSynthesizer(
    group_profiles=group_profiles,
    contracts_per_group=contracts_per_group,
    n_safras=n_safras,
    random_seed=0,
    start_safra = 202001,
    target_ratio=0.10,
    new_contract_rate=0.045,
    closure_rate=0.025,
)

## Geração da safra 0

In [22]:
# 3) Gere os dados
df_snapshot, df_panel, df_trace = synth.generate()

print(f"Snapshot gerado com {len(df_snapshot):,} linhas")
print(f"Painel gerado com {len(df_panel):,} linhas")
print(f"Rastro gerado com {len(df_trace):,} linhas")
display(df_snapshot.head())

Snapshot gerado com 5,000 linhas
Painel gerado com 141,393 linhas
Rastro gerado com 4,188 linhas


,id_contrato,grupo_homogeneo,data_inicio_contrato,data_ref,safra,dias_atraso,nivel_refinanciamento,renda_mensal,tempo_no_endereco_anual,tempo_no_emprego_anual,...,ever360m18,flag_cura,write_off,id_cliente,data_nascimento,sexo,duration_m,age_months,data_fim_contrato,subcluster
0,10000000,GH1,2019-08-05,2020-01-01,202001,15,0,3359.957275,7.178588,4.577875,...,0,0,0,8035.0,1994-05-06,M,65.0,0.0,NaT,4.0
1,10000001,GH1,2019-07-30,2020-01-01,202001,30,0,5109.780273,0.218844,0.054273,...,0,0,0,7908.0,1996-03-20,F,26.0,0.0,NaT,12.0
2,10000002,GH1,2019-11-20,2020-01-01,202001,120,0,2223.669434,0.041022,4.989428,...,0,0,0,1334.0,1969-09-23,F,8.0,0.0,NaT,2.0
3,10000003,GH1,2019-12-15,2020-01-01,202001,15,0,5381.608887,10.507204,3.453141,...,0,0,0,1162.0,1992-05-19,M,32.0,0.0,NaT,12.0
4,10000004,GH1,2019-12-16,2020-01-01,202001,30,0,3203.049072,13.873808,4.244660,...,1,0,0,172.0,1988-05-14,F,44.0,0.0,NaT,4.0


In [23]:
df_panel.groupby('grupo_homogeneo')['ever90m12'].mean()*100

grupo_homogeneo
GH1    56.653435
GH2    56.326473
GH3    56.659142
GH4    55.489614
GH5    56.628449
Name: ever90m12, dtype: float64

In [24]:
df_panel['nivel_refinanciamento'].describe()

count    141393.000000
mean          0.033538
std           0.189603
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           3.000000
Name: nivel_refinanciamento, dtype: float64

In [26]:
# balanceando target
sampler = TargetSampler(target_ratio=0.10)
balanced = sampler.fit_transform(df_panel, target_col="ever90m12")

#prev = balanced.groupby("safra")["ever90m12"].mean()*100
#print("Prev per safra (balanced):")
#print(prev.head())

In [28]:
df_snapshot.groupby('sexo')['ever90m12'].mean()*100

sexo
F    69.123842
M    67.581047
N    59.545455
Name: ever90m12, dtype: float64

In [29]:
df_snapshot.groupby('grupo_homogeneo')['ever90m12'].mean()*100

grupo_homogeneo
GH1    68.6
GH2    66.3
GH3    67.6
GH4    68.5
GH5    68.8
Name: ever90m12, dtype: float64

In [ ]:
balanced.groupby('grupo_homogeneo')['ever90m12'].mean()*100

In [ ]:
len(balanced), len(df_panel)

In [ ]:
balanced['safra'].value_counts()

In [ ]:
balanced.groupby(by='safra', as_index=True)['ever90m12'].mean()*100

### Contagem de contratos por `grupo_homogeneo`

In [ ]:
df_snapshot['grupo_homogeneo'].value_counts()

### Estatísticas descritivas das variáveis numéricas

In [ ]:
cols_numericas = balanced.select_dtypes(include='number').columns.drop(['id_contrato', 'id_cliente'])

agg_df = (
    balanced[cols_numericas.tolist() + ['grupo_homogeneo']]
    .groupby('grupo_homogeneo')
    .agg(['min', 'mean', 'max'])
)

# Flatten columns
agg_df.columns = ['_'.join(col) for col in agg_df.columns]
agg_df.reset_index(inplace=True)

# Supondo que agg_df tenha a coluna 'grupo_homogeneo'
agg_df = agg_df.copy()
agg_df['grupo_idx'] = agg_df['grupo_homogeneo'].str.extract(r'GH(\d+)').astype(int)
agg_df = agg_df.sort_values('grupo_idx').drop(columns='grupo_idx')

agg_df.head(10)

### Distribuição de `dias_atraso`

In [ ]:
plt.figure()
balanced['dias_atraso'].hist(bins=30)
plt.xlabel('Dias em atraso')
plt.ylabel('Número de contratos')
plt.title('Histograma de dias em atraso – safra 0')
plt.show()

In [ ]:
df_snapshot.shape